# Organic Avocado Demand - Data Preprocessing

In [1]:
import pandas as pd
import datetime as dt

In [2]:
data = pd.read_csv('avocado-updated-2020.csv')
data.head()

,date,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,geography
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-04,1.79,1373.95,57.42,153.88,0.00,1162.65,1162.65,0.00,0.0,organic,2015,Albany
2,2015-01-04,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.0,conventional,2015,Atlanta
3,2015-01-04,1.76,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.0,organic,2015,Atlanta
4,2015-01-04,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.0,conventional,2015,Baltimore/Washington


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30021 entries, 0 to 30020
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           30021 non-null  object 
 1   average_price  30021 non-null  float64
 2   total_volume   30021 non-null  float64
 3   4046           30021 non-null  float64
 4   4225           30021 non-null  float64
 5   4770           30021 non-null  float64
 6   total_bags     30021 non-null  float64
 7   small_bags     30021 non-null  float64
 8   large_bags     30021 non-null  float64
 9   xlarge_bags    30021 non-null  float64
 10  type           30021 non-null  object 
 11  year           30021 non-null  int64  
 12  geography      30021 non-null  object 
dtypes: float64(9), int64(1), object(3)
memory usage: 3.0+ MB


In [4]:
data['date'] = pd.to_datetime(data['date'])
data['type'].unique()

array(['conventional', 'organic'], dtype=object)

In [5]:
data['geography'].unique()

array(['Albany', 'Atlanta', 'Baltimore/Washington', 'Boise', 'Boston',
       'Buffalo/Rochester', 'California', 'Charlotte', 'Chicago',
       'Cincinnati/Dayton', 'Columbus', 'Dallas/Ft. Worth', 'Denver',
       'Detroit', 'Grand Rapids', 'Great Lakes', 'Harrisburg/Scranton',
       'Hartford/Springfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'Las Vegas', 'Los Angeles', 'Louisville', 'Miami/Ft. Lauderdale',
       'Midsouth', 'Nashville', 'New Orleans/Mobile', 'New York',
       'Northeast', 'Northern New England', 'Orlando', 'Philadelphia',
       'Phoenix/Tucson', 'Pittsburgh', 'Plains', 'Portland',
       'Raleigh/Greensboro', 'Richmond/Norfolk', 'Roanoke', 'Sacramento',
       'San Diego', 'San Francisco', 'Seattle', 'South Carolina',
       'South Central', 'Southeast', 'Spokane', 'St. Louis', 'Syracuse',
       'Tampa', 'Total U.S.', 'West', 'West Tex/New Mexico'], dtype=object)

In [6]:
# Select US demand for organic avocados
my_data = data[(data['geography'] == 'Total U.S.') &
               (data['type'] == 'organic')][['date', 'total_volume']]
my_data

,date,total_volume
103,2015-01-04,612910.15
211,2015-01-11,669528.88
319,2015-01-18,713120.00
427,2015-01-25,556368.86
535,2015-02-01,740896.97
...,...,...
29584,2020-04-19,2182476.51
29692,2020-04-26,2135665.00
29800,2020-05-03,2157708.00
29908,2020-05-10,2388687.00


In [7]:
# Dates between observations
my_data['date_diff'] = my_data['date'].diff()
my_data['date_diff'].value_counts()

7 days     271
6 days       3
1 days       1
36 days      1
8 days       1
Name: date_diff, dtype: int64

In [8]:
# These look like weekly data
# Print out observations that do not have 7 date_diff 
my_data[my_data['date_diff'] != '7 days']

,date,total_volume,date_diff
103,2015-01-04,612910.15,NaT
17056,2018-01-01,1243940.09,1 days
17164,2018-01-07,1517332.70,6 days
22348,2019-01-07,1706136.92,36 days
22456,2019-01-13,1618207.00,6 days
27964,2020-01-06,1661847.67,8 days
28072,2020-01-12,1960946.80,6 days


In [9]:
# December 2018 is missing
# Drop off 2019 and 2020 observations
my_data = my_data[(my_data['date'].dt.year != 2019) &
                  (my_data['date'].dt.year != 2020)].set_index('date')
my_data.drop('date_diff', axis=1, inplace=True)

In [10]:
# Since we have one date difference between Jan 1st 2018 and the last date of 2017
# Examine last date of 2017 and first date of 2018
print(my_data[my_data.index.year == 2017].tail(1))
print(my_data[my_data.index.year == 2018].head(1))

            total_volume
date                    
2017-12-31    1243940.09
            total_volume
date                    
2018-01-01    1243940.09


In [11]:
# This value is duplicated
# There may be a discrepancy among first days of a week  
my_data['weekday'] = my_data.index.strftime('%a')
my_data['weekday'].value_counts()

Sun    205
Mon      1
Name: weekday, dtype: int64

In [12]:
# Print out the observation starts on Monday
my_data[my_data['weekday'] == 'Mon']

,total_volume,weekday
date,,
2018-01-01,1243940.09,Mon


In [13]:
# Drop the first date of 2018, weekday
# Rename our target
my_data.drop(pd.Timestamp('2018-01-01'), inplace=True)
my_data.drop('weekday', axis=1, inplace=True)
my_data.rename(columns={'date':'ds', 'total_volume':'y'}, inplace=True)
my_data

,y
date,
2015-01-04,612910.15
2015-01-11,669528.88
2015-01-18,713120.00
2015-01-25,556368.86
2015-02-01,740896.97
...,...
2018-11-04,1436243.31
2018-11-11,1534804.73
2018-11-18,1652276.32


In [14]:
my_data.index.nunique()

205

In [15]:
my_data.describe()

,y
count,2.050000e+02
mean,1.075027e+06
std,3.693105e+05
min,5.018149e+05
25%,7.616759e+05
50%,1.045450e+06
75%,1.387187e+06
max,2.114577e+06


In [16]:
# Export to a csv file
my_data.to_csv('organic_avocado.csv', index_label=False)